In [7]:
import panel as pn
import mricloudpy as mp
from tkinter import Tk, filedialog
import ctypes

pn.extension('plotly')
# pn.extension('tabulator')

# Kermit, Miss Piggy, Fozzie, Gonzo, Rowlf, Scooter, Animal, Pepe, Rizzo, Beaker, Statler, Waldorf, Swedish Chef

### Sidebar items

In [8]:
dir_select_button = pn.widgets.Button(name='Select data directory', button_type='primary')
dir_select_text = pn.widgets.StaticText(visible=False)
id_type_select = pn.widgets.Select(name='ID Type', options={'Numeric': 'numeric',
                                                         'Filename': 'filename',
                                                         'Custom': 'custom'})
id_list_text = pn.widgets.TextAreaInput(name='ID List', visible=False, 
                                        placeholder='ex: Kermit, Miss Piggy, Fozzie...')
file_upload_button = pn.widgets.Button(name='Upload', button_type='primary')
data = pn.widgets.Tabulator(height=500, width=300, page_size=100, pagination='local', disabled=True)

### Main items

In [9]:
vis_select = pn.widgets.Select(name='Visual', options={'Sunburst':'sunburst', 
                                                       'Treemap': 'treemap',
                                                       'Icicle': 'icicle',
                                                       'Bar': 'bar',
                                                       'Mean Difference': 'mean_diff',
                                                       'Correlation Matrix': 'corr_matrix'},
                                                       width=140)
type_select = pn.widgets.Select(name='Type', options=['1', '2'], width=50, visible=True)
base_level_select = pn.widgets.Select(name='Base Level', options=['1', '2', '3', '4', '5'], width=50, visible=True)
id_single_select = pn.widgets.Select(name='ID', options=[], width=100, visible=True)
id_multi_text = pn.widgets.TextAreaInput(name='ID List', placeholder='ex: Kermit, Miss Piggy... (or \"all\")',
                                          width=150, visible=False)
x_select = pn.widgets.Select(name='x', options=['ID', 'Object'], width=100, visible=False)
y_select = pn.widgets.Select(name='y', options=['Prop', 'Volume'], width=100, visible=False)
log_select = pn.widgets.Select(name='Logarithm', options=['False', 'True'], width=100, visible=False)
generate_button = pn.widgets.Button(name='Generate', button_type='primary')

vis_plot = pn.pane.Plotly(sizing_mode="stretch_width", config={'responsive': True}, height=700, visible = False)


### Sidebar events

In [10]:
def dir_select(event):
    ctypes.windll.shcore.SetProcessDpiAwareness(1)
    root = Tk()
    root.withdraw()                                        
    root.call('wm', 'attributes', '.', '-topmost', True) 
    # root.call('tk', 'scaling', 2.0)  
    dir_select.dir = filedialog.askdirectory(mustexist=True)
    dir_select_text.value = dir_select.dir
    dir_select_text.visible = True

def check_custom(event):
    if event.new == 'custom':
        id_list_text.visible = True
    else:
        id_list_text.visible = False

def custom_list_format(event):
    text = event.new
    text = text.replace(', ', ',')
    custom_list_format.text_list = text.split(',')

def upload(event):
    upload.is_uploaded = False
    if dir_select.dir is not None:
        try:
            upload.dataset = mp.Data(path=dir_select.dir, id_type=id_type_select.value, 
                             id_list=custom_list_format.text_list)
            data.value = upload.dataset.get_data()
        except:
            upload.dataset = mp.Data(path=dir_select.dir, id_type=id_type_select.value)
            data.value = upload.dataset.get_data()
        data.visible = True
        id_single_select.options = data.value['ID'].unique().tolist()
        upload.is_uploaded = True

dir_select_button.on_click(dir_select)
id_type_select.param.watch(check_custom, 'value')
id_list_text.param.watch(custom_list_format, 'value')
file_upload_button.on_click(upload)
data.visible = False

side_col = pn.Column('# Upload Data',
                     dir_select_button,
                     dir_select_text, 
                     id_type_select,
                     id_list_text, 
                     file_upload_button,
                     pn.layout.Divider(),
                     '### Data Preview',
                     data)

### Main events

In [11]:
def check_vis_type(event):
    if event.new == 'sunburst' or event.new == 'treemap' or event.new == 'icicle':
        type_select.visible = True
        base_level_select.visible = True
        id_single_select.visible = True
        id_multi_text.visible = False
        x_select.visible = False
        y_select.visible = False
        log_select.visible = False
    elif event.new == 'bar':
        type_select.visible = True
        base_level_select.visible = True
        id_single_select.visible = False
        id_multi_text.visible = True
        x_select.visible = True
        y_select.visible = True
        log_select.visible = True
    elif event.new == 'mean_diff':
        type_select.visible = True
        base_level_select.visible = True
        id_single_select.visible = False
        id_multi_text.visible = True
        x_select.visible = True
        y_select.visible = False
        log_select.visible = False
    elif event.new == 'corr_matrix':
        type_select.visible = True
        base_level_select.visible = True
        id_single_select.visible = False
        id_multi_text.visible = True
        x_select.visible = False
        y_select.visible = False
        log_select.visible = False

def id_list_format(event):
    text = event.new
    if text == 'all':
        id_list_format.text_list = list(id_single_select.options)
    else:
        text = text.replace(', ', ',')
        id_list_format.text_list = text.split(',')
    

def generate(event):
    if upload.is_uploaded:
        if vis_select.value == 'sunburst':
            vis_plot.object = upload.dataset.generate_sunburst(type=int(type_select.value), 
                                                        id=str(id_single_select.value), 
                                                        base_level=int(base_level_select.value))
            vis_plot.visible = True
        elif vis_select.value == 'treemap':
            vis_plot.object = upload.dataset.generate_treemap(type=int(type_select.value), 
                                                        id=str(id_single_select.value), 
                                                        base_level=int(base_level_select.value))
            vis_plot.visible = True
        elif vis_select.value == 'icicle':
            vis_plot.object = upload.dataset.generate_icicle(type=int(type_select.value), 
                                                        id=str(id_single_select.value), 
                                                        base_level=int(base_level_select.value))
            vis_plot.visible = True
        elif vis_select.value == 'bar':
            vis_plot.object = upload.dataset.generate_bar(type=int(type_select.value), 
                                                        id=id_list_format.text_list, 
                                                        level=int(base_level_select.value),
                                                        x=x_select.value,
                                                        y=y_select.value,
                                                        log_y=eval(log_select.value))
            vis_plot.visible = True
        elif vis_select.value == 'mean_diff':
            vis_plot.object = upload.dataset.generate_mean_diff(type=int(type_select.value),
                                                                id=id_list_format.text_list,
                                                                level=int(base_level_select.value),
                                                                color=x_select.value)
            vis_plot.visible = True
        elif vis_select.value == 'corr_matrix':
            vis_plot.object = upload.dataset.generate_corr_matrix(type=int(type_select.value),
                                                                  id=id_list_format.text_list,
                                                                  level=int(base_level_select.value))
            vis_plot.visible = True
    

vis_select.param.watch(check_vis_type, 'value')
id_multi_text.param.watch(id_list_format, 'value')
generate_button.on_click(generate)

main_header = pn.Row(vis_select, type_select, base_level_select, id_single_select, 
                     id_multi_text, x_select, y_select, log_select)
main_col = pn.Column('# Main Column',
                     main_header,
                     generate_button,
                     vis_plot)

In [12]:
page = pn.template.BootstrapTemplate(
    title='MRICloudPy',
    sidebar=[side_col],
)

page.main.append(
    pn.Column(main_col)
)

page.show()

Launching server at http://localhost:51693


import_data: Data files found
kirby127a_3_1_ax_283Labels_M2_corrected_stats.txt
kirby142a_3_1_ax_283Labels_M2_corrected_stats.txt
kirby239a_3_1_ax_283Labels_M2_corrected_stats.txt
kirby346a_3_1_ax_283Labels_M2_corrected_stats.txt
kirby422a_3_1_ax_283Labels_M2_corrected_stats.txt
kirby492a_3_1_ax_283Labels_M2_corrected_stats.txt
kirby501a_3_1_ax_283Labels_M2_corrected_stats.txt
kirby505a_3_1_ax_283Labels_M2_corrected_stats.txt
kirby656a_3_1_ax_283Labels_M2_corrected_stats.txt
kirby679a_3_1_ax_283Labels_M2_corrected_stats.txt
kirby742a_3_1_ax_283Labels_M2_corrected_stats.txt
kirby800a_3_1_ax_283Labels_M2_corrected_stats.txt
kirby814a_3_1_ax_283Labels_M2_corrected_stats.txt
import_data: Importing...
import_data: Import successful
generate_mean_diff: Generating...
generate_mean_diff: Generation successful
